In [6]:
import requests
import zipfile
import io
import os
import pandas as pd
import sqlite3

In [11]:
url = "https://archive.ics.uci.edu/static/public/601/ai4i+2020+predictive+maintenance+dataset.zip"
response = requests.get(url)
response.raise_for_status()  # Raise an exception for HTTP errors

zip_file = zipfile.ZipFile(io.BytesIO(response.content))

# Extract the contents to a directory (e.g., 'data')
extract_dir = 'data/raw'
if not os.path.exists(extract_dir):
    os.makedirs(extract_dir)

zip_file.extractall(extract_dir)

print(f"Downloaded and extracted data to '{extract_dir}'")

# Assuming the CSV file is named 'ai4i2020.csv'
csv_file_path = os.path.join(extract_dir, 'ai4i2020.csv')

# Read the CSV file into a Pandas DataFrame
preprocessed_data = pd.read_csv(csv_file_path)

# Prepare data: drop Product ID and remap Type values
db_df = preprocessed_data.drop(columns=['Product ID']).copy()
type_map = {'L': '00', 'M': '01', 'H': '11'}
db_df['Type'] = db_df['Type'].map(type_map)
# Normalize values of 'Air temperature [K]', 'Process temperature [K]', 
min_air_temp, max_air_temp = 300 - 3*2, 300 + 3*2
min_process_temp, max_process_temp = min_air_temp + (10 - 3*1), max_air_temp + (10 + 3*1)
db_df['Air temperature [K]'] = (db_df['Air temperature [K]'] - min_air_temp) / (max_air_temp - min_air_temp)
db_df['Process temperature [K]'] = (db_df['Process temperature [K]'] - min_process_temp) / (max_process_temp - min_process_temp)
# Normalize values of 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]'
db_df['Rotational speed [rpm]'] = (db_df['Rotational speed [rpm]'] - db_df['Rotational speed [rpm]'].min()) / (db_df['Rotational speed [rpm]'].max() - db_df['Rotational speed [rpm]'].min())
db_df['Torque [Nm]'] = (db_df['Torque [Nm]'] - db_df['Torque [Nm]'].min()) / (db_df['Torque [Nm]'].max() - db_df['Torque [Nm]'].min())
db_df['Tool wear [min]'] = (db_df['Tool wear [min]'] - db_df['Tool wear [min]'].min()) / (db_df['Tool wear [min]'].max() - db_df['Tool wear [min]'].min())



# Create a SQLite database and write the table
conn = sqlite3.connect('ai4i2020.db')
db_df.to_sql('machine_data', conn, if_exists='replace', index=False)
conn.close()


Downloaded and extracted data to 'data/raw'


In [9]:
preprocessed_data['Process temperature [K]'].values.std(), preprocessed_data['Process temperature [K]'].values.mean(), preprocessed_data['Process temperature [K]'].values.min(), preprocessed_data['Process temperature [K]'].values.max()

(np.float64(1.483660030600002),
 np.float64(310.00556),
 np.float64(305.7),
 np.float64(313.8))